In [1]:
import numpy as np

# Read the data from the cvs file

In [2]:
data = np.loadtxt('load_data_with_void_markers.csv', 
           dtype={'names': ('point', 'empty', 'x', 'y'), 'formats': 
                  (np.str_, np.str_, np.float_, np.float_)},
                       skiprows=0, delimiter=',', unpack=False)

In [3]:
data[0]

('', '', -1.367, -11.148)

# Identify the missing indexes

In [6]:
idx_data_ = np.array([0,1,2,3,5,6,7,8])

In [7]:
i_grid, j_grid = np.mgrid[0:3, 0:3]

In [8]:
i_grid_f, j_grid_f = i_grid.flatten(), j_grid.flatten()
i_grid_f_, j_grid_f_ = i_grid_f[idx_data_], j_grid_f[idx_data_]
i_grid_f_, j_grid_f

(array([0, 0, 0, 1, 1, 2, 2, 2]), array([0, 1, 2, 0, 1, 2, 0, 1, 2]))

In [9]:
P_IJ = np.array([i_grid_f_, j_grid_f_], dtype=np.int_).T
P_IJ

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])

In [10]:
ij_grid = np.zeros((3,3,), dtype=np.bool_)
ij_grid[i_grid_f_, j_grid_f_] = True
ij_grid

array([[ True,  True,  True],
       [ True, False,  True],
       [ True,  True,  True]])

In [11]:
missing_points = np.where(ij_grid == False)

In [12]:
mp_i, mp_j = missing_points

In [13]:
mp_i

array([1])

In [14]:
interp_idx_nam = np.array([[mp_i - 1, mp_j],
                       [mp_i + 1, mp_j],
                       [mp_i, mp_j - 1],
                       [mp_i, mp_j + 1]]) 
interp_idx_nam

array([[[0],
        [1]],

       [[2],
        [1]],

       [[1],
        [0]],

       [[1],
        [2]]])

# Emulate the GOM data with a missing point 

In [15]:
X, Y = np.mgrid[10:13,20:23] 

In [26]:
gom_data = np.array([X.flatten(), Y.flatten()]).T
XY_Pa = gom_data
gom_data

array([[10, 20],
       [10, 21],
       [10, 22],
       [11, 20],
       [11, 21],
       [11, 22],
       [12, 20],
       [12, 21],
       [12, 22]])

In [17]:
gom_data_ = gom_data[idx_data_]

In [18]:
gom_data_

array([[10, 20],
       [10, 21],
       [10, 22],
       [11, 20],
       [11, 22],
       [12, 20],
       [12, 21],
       [12, 22]])

# Interpolate the data in the missing points

In [19]:
X_IJ, Y_IJ = np.zeros((3,3,), dtype=np.float_), np.zeros((3,3,), dtype=np.float_)
X_IJ[i_grid_f_, j_grid_f_] = gom_data_[:,0]
Y_IJ[i_grid_f_, j_grid_f_] = gom_data_[:,1]
X_IJ, Y_IJ

(array([[10., 10., 10.],
        [11.,  0., 11.],
        [12., 12., 12.]]),
 array([[20., 21., 22.],
        [20.,  0., 22.],
        [20., 21., 22.]]))

In [20]:
ij_anm = np.einsum('nam->anm', interp_idx_nam)
i_nm, j_nm = ij_anm

In [21]:
X_inp = np.average(X_IJ[i_nm, j_nm], axis=-2)
Y_inp = np.average(Y_IJ[i_nm, j_nm], axis=-2)
X_inp, Y_inp

(array([11.]), array([21.]))

In [22]:
X_IJ[ij_grid == False] = X_inp
Y_IJ[ij_grid == False] = Y_inp
X_IJ, Y_IJ

(array([[10., 10., 10.],
        [11., 11., 11.],
        [12., 12., 12.]]),
 array([[20., 21., 22.],
        [20., 21., 22.],
        [20., 21., 22.]]))